# Pytorch Quick Tip

In [6]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
torch.__version__



'1.7.1'

In [9]:
available_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{available_device} is available")
device = torch.device(device=available_device)
device

cuda is available


device(type='cuda')

## 1. Pytorch Quick Tip: Calculate Mean and Standard Deviation of Data
- https://www.youtube.com/watch?v=y6IEcEBRZks&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=45

In [4]:
train_set = datasets.CIFAR10(root="pytorch_dataset",
                           download=True,
                           transform=transforms.ToTensor()
                          )

train_loader = DataLoader(dataset=train_set, shuffle=True, batch_size=64)

print("done")

Files already downloaded and verified
done


In [5]:
def get_mean_std(loader):
    # Var[x] = E[X**2] - E[X]**2
    channels_sum, channels_squared_sum, num_batches = 0, 0 ,0
    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1
    mean = channels_sum/num_batches
    std = (channels_squared_sum/num_batches - mean**2) ** 0.5
    return mean, std

mean, std = get_mean_std(train_loader)
print(mean)
print(std)

tensor([0.4914, 0.4822, 0.4466])
tensor([0.2470, 0.2435, 0.2616])


## 2. Pytorch Quick Tip: Weight Initialization
- https://www.youtube.com/watch?v=xWQ-p_o0Uik&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=46

In [7]:
import torch.nn as nn
import torch.nn.functional as F

In [15]:
# Simple CNN example
class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5,128)
        self.fc2 = nn.Linear(128,num_classes)
    
        self.initialize_weights()
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(x.shape[0], -1) # flatten CONV -> FC
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                    
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
                
        print("done weight initialization")

In [16]:
model = CNN().to(device)

done weight initialization


## 3. Pytorch Quick Tip: Using a Learning Rate Scheduler
- https://www.youtube.com/watch?v=P31hB37g4Ak&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=47

In [ ]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

In [ ]:
## In training loop
for epoch in range(num_epochs):
    losses = []
    for x_train, y_train in train_loader:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        x_train = x_train.float()/255
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        y_pred = cnn_model(x_train)
        
        loss = criterion(y_pred, y_train)
        
        losses.append(loss.item())
        
        loss.backward()
        
        optimizer.step()
        
    mean_loss = sum(losses)/ len(losses)
    scheduler.step(mean_loss)
    
    print(f" loss at the end of {epoch} epoch is {loss}")

print("Done Training")

## 4. Pytorch Quick Tip: Reproducible Results and Deterministic Behavior
- https://www.youtube.com/watch?v=1SZocGaCAr8&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=44

In [18]:
torch.__version__

'1.7.1'

In [17]:
import numpy as np
import random

In [45]:
seed = 42  # Answer to the ultimate question of life
torch.manual_seed(seed)
np.random.seed(seed)

# id using cuda
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

x = torch.rand(5,5)
print(torch.einsum('ii->', x))

tensor(2.3801)


In [46]:
x = torch.rand(5,5)
print(torch.einsum('ii->', x))

tensor(2.3904)


## 5. PyTorch Quick Tip: How to get a Progress Bar
- https://www.youtube.com/watch?v=RKHopFfbPao&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=43
- used tqdm 
- `from tqdm import tqdm`

In [48]:
loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
for batch_idx, (x_train , y_train) in loop :
    '''
            # Other codes 

    '''
    loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
    loop.set_postfix(loss=loss.item(), acc=torch.rand(1).item())
loop